In [1]:
# Start by importing the relevant libraries.
import pandas as pd
import pickle
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
# load delivery dataframe from dataset exported under data_extraction
df = pickle.load(open('dataset_level2_first_innings.pkl', 'rb'))

From here on, we will work on modifying this DataFrame to contain data for the 8 features we want to use for the developement of our model. These features are as follows: batting team, bowling team, current score, wickets left, current run rate, city, balls left, last five overs (runs scored)

In [3]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,1,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,1,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,1,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,1,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,1,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...
115320,963,Sri Lanka,Australia,19.3,1,0,Colombo,R Premadasa Stadium
115321,963,Sri Lanka,Australia,19.4,0,0,Colombo,R Premadasa Stadium
115322,963,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,R Premadasa Stadium
115323,963,Sri Lanka,Australia,19.6,2,0,Colombo,R Premadasa Stadium


2/8 features have been extracted.

In [4]:
# Start by focusing on the city column.
# Count the number of NaN values in the city column.
df['city'].isnull().sum()

12360

In [5]:
# For the line items with NaN city values, look at the venue values.
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        3425
Harare Sports Club                         2731
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sharjah Cricket Stadium                     757
Sydney Cricket Ground                       749
Adelaide Oval                               498
Rawalpindi Cricket Stadium                  368
Sylhet International Cricket Stadium        128
Sylhet Stadium                              121
Carrara Oval                                 64
Name: count, dtype: int64

In [6]:
# The output above indicates that for NaN values in the city column, we can use the first word from the values 
# of the venue column.
# We create a condition using np.where() to assign values where city column value is NaN
# These values are outputted to an array called cities that can then update the DataFrame.
cities = np.where(df['city'].isnull(), df['venue'].str.split().apply(lambda x:x[0]), df['city'])
df['city'] = cities

In [7]:
# Check all columns for any NaN values.
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [8]:
# Remove the Venue column since it is not needed anymore = not a feature, redundant with city.
df.drop(columns = ['venue'], inplace = True)

In [9]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
115320,963,Sri Lanka,Australia,19.3,1,0,Colombo
115321,963,Sri Lanka,Australia,19.4,0,0,Colombo
115322,963,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
115323,963,Sri Lanka,Australia,19.6,2,0,Colombo


In [10]:
# For city column, we also want to trim the data to include cities for which enough data 
# is available to optimally train the model. We define a conditon of atlest 5 matches.
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index.tolist()
df = df[df['city'].isin(eligible_cities)]

In [11]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
115320,963,Sri Lanka,Australia,19.3,1,0,Colombo
115321,963,Sri Lanka,Australia,19.4,0,0,Colombo
115322,963,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
115323,963,Sri Lanka,Australia,19.6,2,0,Colombo


3/8 features have been extracted.

In [12]:
# Now we focus on the current score feature. Currently, our dataframe only has runs conceded
# for every delivery.
# We use groupby here which is an operation to split a python object, apply a funciton and combine
# the results in another python object that we can then run operations on.
df.loc[:,'current_score'] = df.groupby('match_id')['runs'].cumsum()

In [13]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3
...,...,...,...,...,...,...,...,...
115320,963,Sri Lanka,Australia,19.3,1,0,Colombo,125
115321,963,Sri Lanka,Australia,19.4,0,0,Colombo,125
115322,963,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125
115323,963,Sri Lanka,Australia,19.6,2,0,Colombo,127


4/8 features have been extracted.

In [14]:
# Now we focus on the balls left feature.
# Start by splitting the data in 'ball' column into over and ball no.
overs = df['ball'].apply(lambda x:str(x).split(".")[0])
ball_no = df['ball'].apply(lambda x:str(x).split(".")[1])
# We can use this computed data to now compute the balls left for every delivery line item.
balls_bowled = (overs.astype('int')*6) + ball_no.astype('int') # No need to account for extras.
balls_left = 120 - balls_bowled
# Add this to the DataFrame
df.loc[:, 'balls_left'] = balls_left.apply(lambda x:0 if x<0 else x)
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,balls_left
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0,119
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0,118
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1,117
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3,116
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3,115
...,...,...,...,...,...,...,...,...,...
115320,963,Sri Lanka,Australia,19.3,1,0,Colombo,125,3
115321,963,Sri Lanka,Australia,19.4,0,0,Colombo,125,2
115322,963,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,1
115323,963,Sri Lanka,Australia,19.6,2,0,Colombo,127,0


5/8 features have been extracted.

In [15]:
# Now we find the wickets left feature from the data.
# Start by creating a list of 1 and 0 to represent whether a wicket was taken on a ball
# Then group data by matches and find the sum
df.loc[:, 'wickets_left'] = df['player_dismissed'].apply(lambda x:0 if x == '0' else 1)
df.loc[:, 'wickets_left'] = df['wickets_left'].astype('int')
df.loc[:, 'wickets_left'] = df.groupby('match_id')['wickets_left'].cumsum()
df.loc[:, 'wickets_left'] = 10 - df['wickets_left']
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,balls_left,wickets_left
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0,119,10
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0,118,10
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1,117,10
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3,116,10
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3,115,10
...,...,...,...,...,...,...,...,...,...,...
115320,963,Sri Lanka,Australia,19.3,1,0,Colombo,125,3,2
115321,963,Sri Lanka,Australia,19.4,0,0,Colombo,125,2,2
115322,963,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,1,1
115323,963,Sri Lanka,Australia,19.6,2,0,Colombo,127,0,1


6/8 features extracted.

In [16]:
# Now we focus on the current run rate feature.
df.loc[:, 'crr'] = (df['current_score']*6)/balls_bowled
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,balls_left,wickets_left,crr
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0,119,10,0.000000
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0,118,10,0.000000
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1,117,10,2.000000
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3,116,10,4.500000
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3,115,10,3.600000
...,...,...,...,...,...,...,...,...,...,...,...
115320,963,Sri Lanka,Australia,19.3,1,0,Colombo,125,3,2,6.410256
115321,963,Sri Lanka,Australia,19.4,0,0,Colombo,125,2,2,6.355932
115322,963,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,1,1,6.302521
115323,963,Sri Lanka,Australia,19.6,2,0,Colombo,127,0,1,6.350000


7/8 features extracted.

In [17]:
# The last feature is runs scored in last 5 overs.
# Group the delivery by matches.
groups = df.groupby('match_id')
# Get a list of all match ids to iterate through.
match_ids = df['match_id'].unique()
# Create a container to serve as a temp buffer for the values of last 5 overs.
last_five = []
# Iterate through each match
for id in match_ids:
    # Compute the values using a rolling window.
    last_five.extend(groups.get_group(id)['runs'].rolling(window = 30).sum().values.tolist())

In [18]:
# Add these values to the DataFrame
df.loc[:,'last_five'] = last_five

In [19]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,balls_left,wickets_left,crr,last_five
0,1,Australia,Sri Lanka,0.1,0,0,Melbourne,0,119,10,0.000000,NaN
1,1,Australia,Sri Lanka,0.2,0,0,Melbourne,0,118,10,0.000000,NaN
2,1,Australia,Sri Lanka,0.3,1,0,Melbourne,1,117,10,2.000000,NaN
3,1,Australia,Sri Lanka,0.4,2,0,Melbourne,3,116,10,4.500000,NaN
4,1,Australia,Sri Lanka,0.5,0,0,Melbourne,3,115,10,3.600000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
115320,963,Sri Lanka,Australia,19.3,1,0,Colombo,125,3,2,6.410256,32.0
115321,963,Sri Lanka,Australia,19.4,0,0,Colombo,125,2,2,6.355932,32.0
115322,963,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,1,1,6.302521,32.0
115323,963,Sri Lanka,Australia,19.6,2,0,Colombo,127,0,1,6.350000,33.0


8/8 features have been extracted.

In [20]:
# Now we move to the output of the model - the final score.
final_df = df.groupby('match_id')['runs'].sum().reset_index().merge(df, on = 'match_id')

In [21]:
final_df

,match_id,runs_x,batting_team,bowling_team,ball,runs_y,player_dismissed,city,current_score,balls_left,wickets_left,crr,last_five
0,1,168,Australia,Sri Lanka,0.1,0,0,Melbourne,0,119,10,0.000000,NaN
1,1,168,Australia,Sri Lanka,0.2,0,0,Melbourne,0,118,10,0.000000,NaN
2,1,168,Australia,Sri Lanka,0.3,1,0,Melbourne,1,117,10,2.000000,NaN
3,1,168,Australia,Sri Lanka,0.4,2,0,Melbourne,3,116,10,4.500000,NaN
4,1,168,Australia,Sri Lanka,0.5,0,0,Melbourne,3,115,10,3.600000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60013,963,128,Sri Lanka,Australia,19.3,1,0,Colombo,125,3,2,6.410256,32.0
60014,963,128,Sri Lanka,Australia,19.4,0,0,Colombo,125,2,2,6.355932,32.0
60015,963,128,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,1,1,6.302521,32.0
60016,963,128,Sri Lanka,Australia,19.6,2,0,Colombo,127,0,1,6.350000,33.0


In [22]:
# Further clean up final DataFrame to include only relevant features.
final_df = final_df[['batting_team', 'bowling_team', 'city', 'current_score', 'balls_left', 'wickets_left', 'crr', 'last_five', 'runs_x']]

In [23]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
0,Australia,Sri Lanka,Melbourne,0,119,10,0.000000,NaN,168
1,Australia,Sri Lanka,Melbourne,0,118,10,0.000000,NaN,168
2,Australia,Sri Lanka,Melbourne,1,117,10,2.000000,NaN,168
3,Australia,Sri Lanka,Melbourne,3,116,10,4.500000,NaN,168
4,Australia,Sri Lanka,Melbourne,3,115,10,3.600000,NaN,168
...,...,...,...,...,...,...,...,...,...
60013,Sri Lanka,Australia,Colombo,125,3,2,6.410256,32.0,128
60014,Sri Lanka,Australia,Colombo,125,2,2,6.355932,32.0,128
60015,Sri Lanka,Australia,Colombo,125,1,1,6.302521,32.0,128
60016,Sri Lanka,Australia,Colombo,127,0,1,6.350000,33.0,128


In [24]:
# Drop rows with NaN values in the last_five column.
final_df.dropna(inplace = True)

In [25]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
29,Australia,Sri Lanka,Melbourne,43,90,10,8.600000,43.0,168
30,Australia,Sri Lanka,Melbourne,44,89,10,8.516129,44.0,168
31,Australia,Sri Lanka,Melbourne,45,88,10,8.437500,45.0,168
32,Australia,Sri Lanka,Melbourne,45,87,10,8.181818,44.0,168
33,Australia,Sri Lanka,Melbourne,45,86,10,7.941176,42.0,168
...,...,...,...,...,...,...,...,...,...
60013,Sri Lanka,Australia,Colombo,125,3,2,6.410256,32.0,128
60014,Sri Lanka,Australia,Colombo,125,2,2,6.355932,32.0,128
60015,Sri Lanka,Australia,Colombo,125,1,1,6.302521,32.0,128
60016,Sri Lanka,Australia,Colombo,127,0,1,6.350000,33.0,128


In [26]:
# Shuffle the data to reduce chances of a bias.
final_df = final_df.sample(final_df.shape[0])

In [27]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
22713,Australia,England,Manchester,86,35,7,6.070588,30.0,145
57953,England,Sri Lanka,Delhi,161,5,7,8.400000,64.0,171
18502,South Africa,India,Johannesburg,31,86,8,5.470588,29.0,126
36915,New Zealand,Zimbabwe,Harare,94,65,10,10.254545,52.0,187
45449,South Africa,Australia,Centurion,90,26,4,5.744681,22.0,128
...,...,...,...,...,...,...,...,...,...
44562,South Africa,Sri Lanka,Hambantota,134,6,4,7.052632,43.0,145
36899,New Zealand,Zimbabwe,Harare,55,81,10,8.461538,37.0,187
8896,South Africa,Pakistan,Johannesburg,153,9,7,8.270270,53.0,188
58659,England,West Indies,Kolkata,108,40,6,8.100000,60.0,155


In [28]:
# Pickle dump the final DataFrame
import pickle
pickle.dump(final_df, open('dataset_level3_first_innings.pkl','wb'))